
/* PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  


In [180]:
import sqlite3
import pandas as pd

In [134]:
con=sqlite3.connect('sqlite_db_pythonsqlite.db')
rs=con.execute('select distinct * from Members as m inner join Bookings as b \
on m.memid=b.memid inner join Facilities as f on b.facid=f.facid')
df=pd.DataFrame(rs.fetchall())
df.columns=[rs.description[i][0] for i in range(len(rs.description))]
df=df.loc[:,~df.columns.duplicated()]


Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [213]:
q10=[]
for name in df.name.unique():
    f=df[df.name==name]
    rev=0
    for i in f.index:
        s=f.loc[i]
        if s.memid==0:
            rev+=s.guestcost*s.slots
        else:
            rev+=s.membercost*s.slots
    q10.append([name, round(rev,2)])
q10=pd.DataFrame(q10, columns=['Facility Name', 'Total Revenue']).sort_values(by='Total Revenue', ascending=False)
print(q10)

     Facility Name  Total Revenue
1   Massage Room 1        50351.6
8   Massage Room 2        14454.6
6   Tennis Court 2        14310.0
5   Tennis Court 1        13860.0
2     Squash Court        13468.0
7  Badminton Court         1906.5
4       Pool Table          270.0
3    Snooker Table          240.0
0     Table Tennis          180.0


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [214]:
q11=[]
for i in df.memid.unique():
    if i!=0:
        f=df[df.memid==i].iloc[0]
        if f.recommendedby!='':
            q11.append([f.surname+' '+f.firstname, df[df.memid==int(f.recommendedby)].iloc[0].surname, df[df.memid==int(f.recommendedby)].iloc[0].firstname])
q11=pd.DataFrame(q11, columns=['Name', 'Recommended, Surname', 'Recommended, First Name']).sort_values(['Recommended, Surname', 'Recommended, First Name'])
print(q11)

                       Name Recommended, Surname Recommended, First Name
15         Sarwin Ramnaresh                Bader                Florence
14              Coplin Joan                Baker                 Timothy
12          Genting Matthew              Butters                  Gerald
10            Baker Timothy              Farrell                  Jemima
11             Pinker David              Farrell                  Jemima
17         Rumney Henrietta              Genting                 Matthew
16            Jones Douglas                Jones                   David
3                Dare Nancy             Joplette                  Janice
7               Jones David             Joplette                  Janice
20                Hunt John              Purview               Millicent
2                Boothe Tim               Rownam                     Tim
0           Joplette Janice                Smith                  Darren
1            Butters Gerald                Smith   

/* Q12: Find the facilities with their usage by member, but not guests */

In [219]:
q12=pd.DataFrame(df[df.memid!=0].groupby(['name', 'surname', 'firstname']).size().reset_index())
q12.columns=['Facility', 'Member Surname', 'Member First Name', 'Usage Count']
print(q12)

            Facility Member Surname Member First Name  Usage Count
0    Badminton Court          Bader          Florence            9
1    Badminton Court          Baker              Anne           10
2    Badminton Court          Baker           Timothy            7
3    Badminton Court         Boothe               Tim           12
4    Badminton Court        Butters            Gerald           20
..               ...            ...               ...          ...
197   Tennis Court 2          Smith            Darren           19
198   Tennis Court 2          Smith              Jack            1
199   Tennis Court 2          Smith             Tracy            2
200   Tennis Court 2       Stibbons            Ponder           31
201   Tennis Court 2          Tracy            Burton            3

[202 rows x 4 columns]


/* Q13: Find the facilities usage by month, but not guests */

In [216]:
q12=df[df.memid!=0].groupby(['facility', 'month', 'surname', 'firstname', 'name']).size().reset_index()
q12=pd.DataFrame(q12)

In [218]:
df['month']=pd.DatetimeIndex(df['starttime']).month
q13=df[df.memid!=0].groupby([ 'name', 'month', 'surname', 'firstname']).size().reset_index()
q13.columns=['Facility', 'Month', 'Member Surname', 'Member Firstname', 'Usage Count']
print(q13)

            Facility  Month Member Surname Member Firstname  Usage Count
0    Badminton Court      7         Boothe              Tim            1
1    Badminton Court      7        Butters           Gerald            4
2    Badminton Court      7           Dare            Nancy            2
3    Badminton Court      7         Rownam              Tim            1
4    Badminton Court      7          Smith           Darren           38
..               ...    ...            ...              ...          ...
367   Tennis Court 2      9         Rumney        Henrietta            1
368   Tennis Court 2      9         Sarwin        Ramnaresh           11
369   Tennis Court 2      9          Smith           Darren            3
370   Tennis Court 2      9          Smith             Jack            1
371   Tennis Court 2      9       Stibbons           Ponder           14

[372 rows x 5 columns]
